# Problems with single-pass multiple prediction and padded convolutions

I explain how the net works that makes predictions for neighbouring samples in one pass. Then I illustrate why this only works for convolutions without padding, atleast the way I do it at the moment.

## Without multiple prediction network

Before, let's look at how you would make predictions for samples if you would use a "normal" network.

Lets take a very simple network with 1d-input:

|Layer|Input Size|Filter Size|Weights|
|-|
|Conv|1x4x1|1x3x1|1 1 1|
|"Dense"|1x2x1|1x2x1|1 1|

Both layers just have one unit.
We get the input 1,2,3,4,5.
We cut this to windows of 1,2,3,4 and 2,3,4,5 to make our predictions.

If you look at the figure below, you will see that there is one computation, namely multiplication of [2 3 4] and [1 1 1], which happens in both passes. These are the computations I try to avoid.

In [3]:
%%html
<svg width="640" height="480" xmlns="http://www.w3.org/2000/svg" xmlns:svg="http://www.w3.org/2000/svg"><!-- Created with SVG-edit - http://svg-edit.googlecode.com/ --><g><title>Layer 1</title><text fill="#000000" stroke="#000000" stroke-width="0" x="241.80556" y="110.3634" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_1">2 3 4 5</text><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" x="178.55556" y="71.97367" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_3">1 2 3 4 5</text><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" x="108.25" y="106.48666" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_4">1 2 3 4</text><text style="cursor: move;" fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="122.18179" y="227.14184" id="svg_6" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)">1 1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" id="svg_15" d="m143.99997,236.8125c-0.03215,1.91251 -0.3199,2.69525 -0.49425,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30867,1.70767 -0.32327,2.72025c-0.00981,0.68097 -0.08562,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29689,1.62439 -0.32072,2.51448c-0.02713,1.01212 -0.00253,1.81076 -0.00253,2.75876c0,0.948 0,1.74631 0,2.58675c0,0.81235 0.04698,1.78174 -0.00075,2.70074c-0.04694,0.90356 -0.45073,1.67355 -0.55003,2.57578c-0.10564,0.95972 0.11478,1.88284 -0.04387,2.68762c-0.16417,0.83282 -0.42821,1.73193 -0.65485,2.56735c-0.24022,0.88553 -0.63937,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11911,1.91278 -0.16049,2.74802c-0.04698,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12663,0.87173 -0.03215,1.58719 -0.03215,2.67621c0,0.77457 0,1.80551 0,2.61252c0,0.89926 0,1.79929 0,2.69925c0,0.89999 0,1.75201 0,2.70074c0,0.81638 0,1.77902 0,2.69925c0,0.84375 -0.00453,1.74377 0,2.58752c0.00507,0.9436 0.64778,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38011,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19533,-1.19531 1.97746,-1.97745c0.54207,-0.54208 1.09538,-1.0954 1.9366,-1.93661c0.56026,-0.56024 1.10699,-1.10699 1.62675,-1.62674c0.47925,-0.47925 1.42201,-1.42651 1.9335,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06174,-1.69763c0.76572,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.64377,-0.33591 1.37488,-0.87704 2.14059,-1.43588c0.68155,-0.49741 1.39444,-1.03616 2.05162,-1.6875c0.66707,-0.66113 1.25513,-1.25513 1.88286,-1.88287c0.627,-0.62698 1.58316,-2.2666 2.0625,-1.59375c0.49626,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55849,0.64609 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.9054 -1.31387,1.46774 -2.09027,2.25c-0.52153,0.52545 -1.20468,1.23346 -1.80469,1.79297c-0.71034,0.66238 -1.27666,1.08246 -1.86057,1.54904c-0.90965,0.72687 -1.45653,1.26782 -2.29041,1.92667c-0.57997,0.45822 -1.32985,1.05646 -2.10684,1.57507c-0.66531,0.44409 -1.52319,0.73419 -2.26866,1.30576c-0.68562,0.52573 -1.28014,1.23956 -1.94475,1.87387c-0.5206,0.49689 -1.35649,0.95886 -2.09109,1.55139c-0.67581,0.5451 -1.04337,1.41602 -1.92599,1.85776c-0.8183,0.40961 -1.83745,0.38681 -2.65622,0.13782c-0.83701,-0.25461 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28989,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80875,-0.659 -1.47195,-1.09024 -2.20407,-1.54901c-0.48975,-0.30692 -1.39177,-0.862 -2.14799,-1.35825c-0.85933,-0.56393 -1.57955,-1.05087 -2.229,-1.5c-0.81497,-0.5636 -1.45012,-1.02014 -2.14941,-1.49899c-0.7275,-0.49817 -1.08913,-0.75778 -2.20256,-1.42648c-0.69903,-0.41983 -1.70811,-0.95108 -2.33427,-1.26852c-0.85784,-0.43491 -1.63166,-0.77744 -2.42625,-1.23825c-0.87508,-0.50748 -1.62825,-1.06125 -2.3954,-1.45493l-0.73959,-0.34583"/><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="101.91353" y="341.5197" id="svg_16" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)">6</text><text style="cursor: move;" fill="#7f3f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="143.37538" y="344.41967" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" id="svg_17" transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)">9</text><text style="cursor: move;" fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="329.57374" y="233.3002" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_18">1 1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m450.0296,246.23123c-0.03214,1.91251 -0.31989,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.69759 -0.30869,1.70766 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32071,2.51447c-0.02716,1.01212 -0.00256,1.81076 -0.00256,2.75876c0,0.948 0,1.74631 0,2.58676c0,0.81235 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10562,0.95972 0.11481,1.88284 -0.04385,2.68762c-0.16415,0.83282 -0.42819,1.73193 -0.65485,2.56735c-0.2402,0.88553 -0.63937,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11911,1.91278 -0.16049,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58719 -0.03217,2.67621c0,0.77457 0,1.80551 0,2.61252c0,0.89926 0,1.79929 0,2.69925c0,0.89999 0,1.75201 0,2.70074c0,0.81638 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.56027,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.9335,-1.9335c0.66733,-0.66144 1.38019,-1.20023 2.06174,-1.69763c0.76572,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.64377,-0.33591 1.37488,-0.87704 2.14059,-1.43588c0.68155,-0.49741 1.39444,-1.03616 2.05161,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62698 1.58316,-2.2666 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64609 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.9054 -1.31387,1.46774 -2.09027,2.25c-0.52151,0.52545 -1.20465,1.23346 -1.80469,1.79297c-0.71033,0.66238 -1.27664,1.08246 -1.86053,1.54904c-0.90967,0.72687 -1.45654,1.26782 -2.29041,1.92667c-0.57999,0.45822 -1.32986,1.05643 -2.10687,1.57507c-0.66528,0.44409 -1.52319,0.73419 -2.26865,1.30576c-0.68564,0.52573 -1.28012,1.23956 -1.94473,1.87387c-0.52063,0.49686 -1.35651,0.95886 -2.09113,1.55139c-0.67578,0.5451 -1.04333,1.41599 -1.92596,1.85776c-0.8183,0.40961 -1.83746,0.38681 -2.65625,0.13782c-0.83698,-0.25461 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71085 -1.28989,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47195,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57956,-1.05087 -2.229,-1.5c-0.81497,-0.5636 -1.4501,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20255,-1.42648c-0.69904,-0.41983 -1.7081,-0.95108 -2.33429,-1.26852c-0.85782,-0.43491 -1.63165,-0.77744 -2.42624,-1.23825c-0.87509,-0.50748 -1.62827,-1.06125 -2.39542,-1.45493l-0.73959,-0.34583" id="svg_19"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="317.49982" y="346.39694" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_20">12</text><text style="cursor: move;" fill="#7f3f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="323.41485" y="350.34136" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)" id="svg_21">9</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="114.45192" y="333.88088" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_22">1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m127.47263,383.29883c-0.03214,1.91251 -0.31992,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30869,1.70767 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02713,1.01212 -0.00253,1.81076 -0.00253,2.75876c0,0.948 0,1.74631 0,2.58676c0,0.81235 0.04697,1.78174 -0.00076,2.70074c-0.04694,0.90356 -0.45071,1.67355 -0.55002,2.57578c-0.10562,0.95972 0.11478,1.88284 -0.04385,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.04697,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12662,0.87173 -0.03214,1.58719 -0.03214,2.67621c0,0.7746 0,1.80551 0,2.61252c0,0.89926 0,1.79929 0,2.69928c0,0.89996 0,1.75198 0,2.70071c0,0.81641 0,1.77902 0,2.69925c0,0.84375 -0.00455,1.74377 0,2.58752c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60263,-0.57581 1.3801,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99218,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97746,-1.97745c0.54207,-0.54208 1.0954,-1.0954 1.9366,-1.93661c0.56027,-0.56024 1.10699,-1.10699 1.62674,-1.62674c0.47925,-0.47925 1.42203,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76576,-0.55881 1.49881,-1.09619 2.36543,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05161,-1.6875c0.66708,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62698,-0.62695 1.58316,-2.2666 2.0625,-1.59375c0.49625,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.5585,0.64609 -1.14902,1.38821 -1.56601,1.875c-0.77557,0.90543 -1.31384,1.46774 -2.09024,2.25c-0.52151,0.52545 -1.20468,1.23346 -1.80469,1.79297c-0.71036,0.66238 -1.27664,1.08246 -1.86058,1.54904c-0.90967,0.72687 -1.45653,1.26782 -2.29041,1.92667c-0.57996,0.45822 -1.32983,1.05646 -2.10684,1.57507c-0.6653,0.44409 -1.52319,0.73419 -2.26865,1.30579c-0.68564,0.5257 -1.28015,1.23953 -1.94476,1.87387c-0.5206,0.49686 -1.35648,0.95883 -2.09108,1.55136c-0.67582,0.5451 -1.04337,1.41602 -1.92599,1.85779c-0.8183,0.40958 -1.83746,0.38681 -2.65622,0.13779c-0.83701,-0.25458 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28989,-1.33398 -2.00449,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13223,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80875,-0.659 -1.47195,-1.09024 -2.20407,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.22897,-1.5c-0.81497,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72752,-0.49817 -1.08914,-0.75778 -2.20255,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73959,-0.34583" id="svg_23"/><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="108.41228" y="435.85915" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_24">15</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="339.6829" y="336.56903" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_26">1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m459.82568,387.41013c-0.0321,1.91251 -0.31989,2.69525 -0.4942,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.1499,0.6976 -0.30872,1.70767 -0.3233,2.72025c-0.00977,0.68097 -0.08557,1.78275 -0.2392,2.60178c-0.11865,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02716,1.01215 -0.00256,1.81076 -0.00256,2.75876c0,0.94803 0,1.74634 0,2.58676c0,0.81238 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10559,0.95975 0.11481,1.88284 -0.04382,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25891,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58722 -0.03217,2.67624c0,0.77457 0,1.80548 0,2.61252c0,0.89926 0,1.79926 0,2.69925c0,0.89996 0,1.75198 0,2.70071c0,0.81641 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.64783,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19537,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.5603,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05157,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62695 1.58319,-2.26657 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64612 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.90543 -1.31384,1.46774 -2.09027,2.25c-0.52148,0.52545 -1.20465,1.23346 -1.80469,1.793c-0.71033,0.66235 -1.27661,1.08246 -1.86053,1.54901c-0.90967,0.7269 -1.45654,1.26782 -2.29041,1.9267c-0.57996,0.45822 -1.32983,1.05643 -2.10687,1.57504c-0.66528,0.44409 -1.52319,0.73419 -2.26862,1.30579c-0.68567,0.5257 -1.28015,1.23956 -1.94476,1.87387c-0.52063,0.49686 -1.35651,0.95886 -2.09113,1.55139c-0.67578,0.54507 -1.04333,1.41599 -1.92596,1.85776c-0.8183,0.40958 -1.83746,0.38681 -2.65625,0.13779c-0.83698,-0.25458 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28986,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.6543,-0.49887 -1.39612,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47192,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.229,-1.5c-0.81494,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20251,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73962,-0.34583" id="svg_27"/><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="299.3386" y="438.73142" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_28">21</text></g></svg>

# With multiple prediction network


We can transform our model to make all the necessary computations in one pass, without duplicate computations.

We enlarge the input and change the dense layer into a convolutional layer.

|Layer|Input Size|Filter Size|Weights|
|-|
|Conv|1x5x1|1x3x1|1 1 1|
|Conv|1x3x1|1x2x1|1 1|

In [4]:
%%html
<svg width="640" height="480" xmlns="http://www.w3.org/2000/svg" xmlns:svg="http://www.w3.org/2000/svg"><!-- Created with SVG-edit - http://svg-edit.googlecode.com/ --><g><title>Layer 1</title><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" x="178.55556" y="71.97367" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_3">1 2 3 4 5</text><text style="cursor: move;" fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="236.71071" y="171.56492" id="svg_6" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.85577, 0, 0, 1.92896, -138.874, -191.144)">1 1 1</text><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="182.33906" y="284.93066" id="svg_16" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)">6</text><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="239.13932" y="293.49306" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" id="svg_17" transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)">9</text><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="241.66999" y="284.91737" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_20">12</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m306.47263,151.29883c-0.03214,1.91251 -0.31992,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30869,1.70767 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32074,2.51448c-0.02713,1.01212 -0.00253,1.81076 -0.00253,2.75876c0,0.948 0,1.74631 0,2.58676c0,0.81235 0.04697,1.78174 -0.00076,2.70073c-0.04694,0.90356 -0.45071,1.67355 -0.55002,2.57579c-0.10562,0.95972 0.11478,1.88283 -0.04385,2.68762c-0.16418,0.83281 -0.42822,1.73192 -0.65485,2.56734c-0.24023,0.88554 -0.6394,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11914,1.91278 -0.16052,2.74803c-0.04697,0.94835 -0.38599,1.66208 -0.5296,2.65076c-0.12662,0.87172 -0.03214,1.58719 -0.03214,2.67621c0,0.77457 0,1.80551 0,2.61252c0,0.89926 0,1.79926 0,2.69925c0,0.89999 0,1.75201 0,2.70074c0,0.81638 0,1.77902 0,2.69925c0,0.84375 -0.00455,1.74376 0,2.58751c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60263,-0.57581 1.3801,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97745,-1.97745c0.54208,-0.54208 1.0954,-1.0954 1.93661,-1.93661c0.56027,-0.56024 1.10699,-1.10699 1.62674,-1.62674c0.47925,-0.47925 1.42203,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49881,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87703 2.14063,-1.43587c0.68152,-0.49742 1.39441,-1.03616 2.05164,-1.6875c0.66705,-0.66113 1.25513,-1.25514 1.88284,-1.88287c0.62698,-0.62698 1.58316,-2.2666 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64609 -1.14902,1.38821 -1.56601,1.875c-0.77554,0.9054 -1.31381,1.46774 -2.09024,2.25c-0.52148,0.52545 -1.20465,1.23346 -1.80469,1.79297c-0.71036,0.66238 -1.27664,1.08246 -1.86057,1.54904c-0.90967,0.72687 -1.45654,1.26782 -2.29041,1.92667c-0.57996,0.45822 -1.32983,1.05643 -2.10684,1.57507c-0.66531,0.44409 -1.52319,0.73419 -2.26865,1.30576c-0.68564,0.5257 -1.28015,1.23956 -1.94476,1.87387c-0.5206,0.49686 -1.35648,0.95886 -2.09109,1.55139c-0.67581,0.5451 -1.04337,1.41599 -1.92599,1.85776c-0.8183,0.40958 -1.83743,0.38681 -2.65622,0.13779c-0.83701,-0.25458 -1.39453,-1.10156 -1.94528,-1.65233c-0.71085,-0.71083 -1.28992,-1.33398 -2.00452,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13223,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80875,-0.659 -1.47192,-1.09024 -2.20407,-1.54901c-0.48975,-0.30692 -1.39175,-0.862 -2.14801,-1.35825c-0.85928,-0.56393 -1.57953,-1.05087 -2.22894,-1.5c-0.81497,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72751,-0.49817 -1.08914,-0.75778 -2.20255,-1.42648c-0.69907,-0.41983 -1.70813,-0.95108 -2.33432,-1.26852c-0.85785,-0.43491 -1.63162,-0.77744 -2.42627,-1.23824c-0.87503,-0.50749 -1.6282,-1.06126 -2.39539,-1.45494l-0.73959,-0.34583" id="svg_23"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="187.11445" y="407.21526" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_24">15</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="231.25317" y="287.53051" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_26">1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m305.82568,326.41013c-0.0321,1.91251 -0.31989,2.69525 -0.4942,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.1499,0.6976 -0.30872,1.70767 -0.3233,2.72025c-0.00977,0.68097 -0.08557,1.78275 -0.2392,2.60178c-0.11865,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02716,1.01215 -0.00256,1.81076 -0.00256,2.75876c0,0.94803 0,1.74634 0,2.58676c0,0.81238 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10559,0.95975 0.11481,1.88284 -0.04382,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25891,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58722 -0.03217,2.67624c0,0.77457 0,1.80548 0,2.61252c0,0.89926 0,1.79926 0,2.69925c0,0.89996 0,1.75198 0,2.70071c0,0.81641 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.64783,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19537,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.5603,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05157,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62695 1.58319,-2.26657 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64612 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.90543 -1.31384,1.46774 -2.09027,2.25c-0.52148,0.52545 -1.20465,1.23346 -1.80469,1.793c-0.71033,0.66235 -1.27661,1.08246 -1.86053,1.54901c-0.90967,0.7269 -1.45654,1.26782 -2.29041,1.9267c-0.57996,0.45822 -1.32983,1.05643 -2.10687,1.57504c-0.66528,0.44409 -1.52319,0.73419 -2.26862,1.30579c-0.68567,0.5257 -1.28015,1.23956 -1.94476,1.87387c-0.52063,0.49686 -1.35651,0.95886 -2.09113,1.55139c-0.67578,0.54507 -1.04333,1.41599 -1.92596,1.85776c-0.8183,0.40958 -1.83746,0.38681 -2.65625,0.13779c-0.83698,-0.25458 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28986,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.6543,-0.49887 -1.39612,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47192,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.229,-1.5c-0.81494,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20251,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73962,-0.34583" id="svg_27"/><text style="cursor: move;" fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="232.12581" y="408.69028" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_28">21</text></g></svg>

This is already the whole idea, it gets a little more complicated if the model has strides somewhere, but it's still possible to do the same trick then. This can speed everything up a lot: in the first layer, for the next neighbour(sample), this avoids all computations except one.

## Same Convolutions (with padding)

With padding, unfortunately we get "unique" computations inbetween neighbouring windows. So we have computations which are not shared and inbetween the windows. See the red numbers below.

In [10]:
%%html
<svg width="640" height="480" xmlns="http://www.w3.org/2000/svg" xmlns:svg="http://www.w3.org/2000/svg"><!-- Created with SVG-edit - http://svg-edit.googlecode.com/ --><g><title>Layer 1</title><text fill="#000000" stroke="#000000" stroke-width="0" x="241.80556" y="110.3634" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_1">2 3 4 5</text><text fill="#000000" stroke="#000000" stroke-width="0" x="178.55556" y="71.97367" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_3">1 2 3 4 5</text><text fill="#000000" stroke="#000000" stroke-width="0" x="108.25" y="106.48666" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" id="svg_4">1 2 3 4</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="122.18179" y="227.14184" id="svg_6" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)">1 1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" id="svg_15" d="m143.99997,236.8125c-0.03215,1.91251 -0.3199,2.69525 -0.49425,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30867,1.70767 -0.32327,2.72025c-0.00981,0.68097 -0.08562,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29689,1.62439 -0.32072,2.51448c-0.02713,1.01213 -0.00253,1.81076 -0.00253,2.75876c0,0.948 0,1.74631 0,2.58675c0,0.81235 0.04698,1.78174 -0.00075,2.70074c-0.04694,0.90356 -0.45073,1.67355 -0.55003,2.57578c-0.10564,0.95972 0.11478,1.88284 -0.04387,2.68762c-0.16417,0.83282 -0.42821,1.73193 -0.65485,2.56735c-0.24022,0.88553 -0.63937,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11911,1.91278 -0.16049,2.74802c-0.04698,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12663,0.87173 -0.03215,1.58719 -0.03215,2.67621c0,0.77457 0,1.80551 0,2.61252c0,0.89926 0,1.79941 0,2.69925c0,0.89999 0,1.75201 0,2.70074c0,0.81638 0,1.77902 0,2.69925c0,0.84375 -0.00453,1.74377 0,2.58752c0.00507,0.9436 0.64778,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38011,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19533,-1.19531 1.97746,-1.97745c0.54207,-0.54208 1.09538,-1.0954 1.9366,-1.93661c0.56026,-0.56024 1.10699,-1.10699 1.62675,-1.62674c0.47925,-0.47925 1.42201,-1.42651 1.9335,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06174,-1.69763c0.76572,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.64377,-0.33591 1.37488,-0.87704 2.14059,-1.43588c0.68155,-0.49741 1.39444,-1.03616 2.05162,-1.6875c0.66707,-0.66113 1.25513,-1.25513 1.88286,-1.88287c0.627,-0.62698 1.58316,-2.2666 2.0625,-1.59375c0.49626,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55849,0.64609 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.9054 -1.31387,1.46774 -2.09027,2.25c-0.52153,0.52545 -1.20468,1.23346 -1.80469,1.79297c-0.71034,0.66238 -1.27667,1.08246 -1.86058,1.54904c-0.90965,0.72687 -1.45654,1.26782 -2.29042,1.92667c-0.57997,0.45822 -1.32985,1.05649 -2.10684,1.57507c-0.6653,0.44409 -1.52318,0.73419 -2.26865,1.30576c-0.68562,0.52585 -1.28015,1.23956 -1.94476,1.87387c-0.5206,0.49698 -1.35649,0.95886 -2.09108,1.55139c-0.67581,0.5451 -1.04338,1.41611 -1.92598,1.85776c-0.8183,0.40973 -1.83745,0.38681 -2.65622,0.13794c-0.83702,-0.25473 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71088 -1.28989,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80876,-0.659 -1.47195,-1.09024 -2.20407,-1.54901c-0.48976,-0.30692 -1.39177,-0.862 -2.14799,-1.35825c-0.85933,-0.56393 -1.57955,-1.05087 -2.229,-1.5c-0.81496,-0.5636 -1.45012,-1.02014 -2.14941,-1.49899c-0.7275,-0.49817 -1.08913,-0.75778 -2.20256,-1.42648c-0.69903,-0.41983 -1.70811,-0.95108 -2.33427,-1.26852c-0.85784,-0.43491 -1.63166,-0.77744 -2.42625,-1.23825c-0.87508,-0.50748 -1.62825,-1.06125 -2.3954,-1.45493l-0.73959,-0.34583"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="101.91353" y="341.5197" id="svg_16" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)">6</text><text fill="#7f3f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="143.37538" y="344.41967" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" id="svg_17" transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)">9</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="329.57374" y="233.3002" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_18">1 1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m450.0296,246.23123c-0.03214,1.91251 -0.31989,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.69759 -0.30869,1.70766 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32071,2.51447c-0.02716,1.01212 -0.00256,1.81076 -0.00256,2.75876c0,0.948 0,1.74631 0,2.58676c0,0.81235 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10562,0.95972 0.11481,1.88284 -0.04385,2.68762c-0.16415,0.83282 -0.42819,1.73193 -0.65485,2.56735c-0.2402,0.88553 -0.63937,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11911,1.91278 -0.16049,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58719 -0.03217,2.67621c0,0.77457 0,1.80551 0,2.61252c0,0.89926 0,1.79932 0,2.69925c0,0.89999 0,1.75201 0,2.70074c0,0.81638 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.56027,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.9335,-1.9335c0.66733,-0.66144 1.38019,-1.20023 2.06174,-1.69763c0.76572,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.64377,-0.33591 1.37488,-0.87704 2.14059,-1.43588c0.68155,-0.49741 1.39444,-1.03616 2.05161,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62698 1.58316,-2.2666 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64609 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.9054 -1.31387,1.46774 -2.09027,2.25c-0.52151,0.52545 -1.20465,1.23346 -1.80469,1.79297c-0.71033,0.66238 -1.27664,1.08246 -1.86053,1.54904c-0.90967,0.72687 -1.45654,1.26782 -2.29041,1.92667c-0.57999,0.45822 -1.32986,1.05643 -2.10687,1.57507c-0.66528,0.44409 -1.52319,0.73419 -2.26865,1.30576c-0.68564,0.52582 -1.28012,1.23956 -1.94473,1.87387c-0.52063,0.49686 -1.35651,0.95886 -2.09113,1.55139c-0.67578,0.5451 -1.04333,1.41599 -1.92596,1.85776c-0.8183,0.40964 -1.83746,0.38681 -2.65625,0.13785c-0.83698,-0.25464 -1.39453,-1.10147 -1.94531,-1.65225c-0.71082,-0.71097 -1.28989,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47195,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57956,-1.05087 -2.229,-1.5c-0.81497,-0.5636 -1.4501,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20255,-1.42648c-0.69904,-0.41983 -1.7081,-0.95108 -2.33429,-1.26852c-0.85782,-0.43491 -1.63165,-0.77744 -2.42624,-1.23825c-0.87509,-0.50748 -1.62827,-1.06125 -2.39542,-1.45493l-0.73959,-0.34583" id="svg_19"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="317.49982" y="346.39694" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_20">12</text><text fill="#7f3f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="323.41485" y="350.34136" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)" id="svg_21">9</text><text fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="114.45192" y="333.88088" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" id="svg_22">1 1 1 1</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m127.47263,383.29883c-0.03214,1.91251 -0.31992,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30869,1.70767 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02713,1.01212 -0.00253,1.81076 -0.00253,2.75876c0,0.948 0,1.74631 0,2.58676c0,0.81235 0.04697,1.78174 -0.00076,2.70074c-0.04694,0.90356 -0.45071,1.67355 -0.55002,2.57578c-0.10562,0.95972 0.11478,1.88284 -0.04385,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.04697,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12662,0.87173 -0.03214,1.58719 -0.03214,2.67621c0,0.77466 0,1.80551 0,2.61252c0,0.89926 0,1.79941 0,2.69928c0,0.89996 0,1.75198 0,2.70071c0,0.81641 0,1.77902 0,2.69925c0,0.84375 -0.00455,1.74377 0,2.58752c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60263,-0.57581 1.3801,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99218,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97746,-1.97745c0.54207,-0.54208 1.0954,-1.0954 1.9366,-1.93661c0.56026,-0.56024 1.10698,-1.10699 1.62674,-1.62674c0.47925,-0.47925 1.42203,-1.42651 1.93352,-1.9335c0.66731,-0.66144 1.3802,-1.20023 2.06172,-1.69763c0.76576,-0.55881 1.49881,-1.09619 2.36543,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05161,-1.6875c0.66708,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62698,-0.62695 1.58316,-2.2666 2.0625,-1.59375c0.49625,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.5585,0.64609 -1.14902,1.38821 -1.56601,1.875c-0.77557,0.90549 -1.31384,1.46774 -2.09024,2.25c-0.52151,0.52545 -1.20468,1.23346 -1.80469,1.79297c-0.71036,0.66238 -1.27664,1.08246 -1.86058,1.54904c-0.90967,0.72687 -1.45653,1.26782 -2.29042,1.92667c-0.57996,0.45822 -1.32982,1.05658 -2.10683,1.57507c-0.6653,0.44409 -1.52319,0.73419 -2.26866,1.30582c-0.68564,0.52579 -1.28015,1.2395 -1.94475,1.87384c-0.5206,0.49698 -1.35649,0.95883 -2.09108,1.55136c-0.67583,0.5451 -1.04337,1.41614 -1.92599,1.85782c-0.8183,0.40967 -1.83746,0.38681 -2.65622,0.13788c-0.83701,-0.25461 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28989,-1.33398 -2.00449,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13223,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80875,-0.659 -1.47195,-1.09024 -2.20407,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.22897,-1.5c-0.81497,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72752,-0.49817 -1.08914,-0.75778 -2.20255,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73959,-0.34583" id="svg_23"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="108.41228" y="435.85915" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_24">25</text><path fill="none" stroke="#000000" stroke-width="4" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" d="m459.82568,387.41013c-0.0321,1.91251 -0.31989,2.69525 -0.4942,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.1499,0.6976 -0.30872,1.70767 -0.3233,2.72025c-0.00977,0.68097 -0.08557,1.78275 -0.2392,2.60178c-0.11865,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02716,1.01227 -0.00256,1.81076 -0.00256,2.75876c0,0.94806 0,1.74643 0,2.58676c0,0.81238 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10559,0.95978 0.11481,1.88284 -0.04382,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25891,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58722 -0.03217,2.67624c0,0.77469 0,1.80548 0,2.61264c0,0.89917 0,1.79926 0,2.69925c0,0.89984 0,1.75186 0,2.70059c0,0.81653 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.64783,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19537,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.5603,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05157,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62683 1.58319,-2.26654 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64618 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.90555 -1.31384,1.46774 -2.09027,2.25c-0.52148,0.52545 -1.20465,1.23346 -1.80469,1.793c-0.71033,0.66235 -1.27661,1.08255 -1.86053,1.54901c-0.90967,0.72693 -1.45654,1.26782 -2.29041,1.92682c-0.57996,0.45813 -1.32983,1.05643 -2.10687,1.57492c-0.66528,0.44409 -1.52319,0.73419 -2.26862,1.30591c-0.68567,0.5257 -1.28015,1.23944 -1.94476,1.87387c-0.52063,0.49686 -1.35651,0.95874 -2.09113,1.5513c-0.67578,0.54504 -1.04333,1.41608 -1.92596,1.85785c-0.8183,0.40958 -1.83746,0.38681 -2.65625,0.13779c-0.83698,-0.25458 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28986,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.6543,-0.49887 -1.39612,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47192,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.229,-1.5c-0.81494,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20251,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73962,-0.34583" id="svg_27"/><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="299.3386" y="438.73142" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_28">35</text><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="81.98675" y="340.84527" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_2">3</text><text fill="#ff0000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="141.15699" y="342.94117" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_5">7</text><text fill="#ff0000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="265.81662" y="348.53024" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_7">5</text><text fill="#000000" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="347.39111" y="347.13299" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" id="svg_8">9</text><text id="svg_9" fill="#007f00" stroke="#000000" stroke-width="0" stroke-dasharray="null" stroke-linejoin="null" stroke-linecap="null" x="345.54279" y="335.84242" font-size="24" font-family="serif" text-anchor="middle" xml:space="preserve" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)">1 1 1 1</text></g></svg>

Now if we do the same transformation as before, we do not get the same numbers anymore (see numbers in red).

In [8]:
%%html
<svg width="640" height="480" xmlns="http://www.w3.org/2000/svg" xmlns:svg="http://www.w3.org/2000/svg"><!-- Created with SVG-edit - http://svg-edit.googlecode.com/ --><g><title>Layer 1</title><text id="svg_3" transform="matrix(2.25, 0, 0, 2.48276, -91.25, -106.759)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="71.97367" x="178.55556" stroke-width="0" stroke="#000000" fill="#000000">1 2 3 4 5</text><text transform="matrix(1.85577, 0, 0, 1.92896, -138.874, -191.144)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" id="svg_6" y="171.56492" x="236.71071" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#007f00">1 1 1</text><text transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" id="svg_16" y="284.93066" x="182.33906" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#000000">6</text><text transform="matrix(1.61856, 0, 0, 1.59052, -84.4059, -213.935)" id="svg_17" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="293.49306" x="239.13932" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#000000">9</text><text id="svg_20" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="284.91737" x="241.66999" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#000000">12</text><path id="svg_23" d="m306.47263,151.29883c-0.03214,1.91251 -0.31992,2.69525 -0.49423,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.14987,0.6976 -0.30869,1.70767 -0.32327,2.72025c-0.0098,0.68097 -0.0856,1.78275 -0.23923,2.60178c-0.11862,0.63232 -0.29688,1.62439 -0.32074,2.51448c-0.02713,1.01213 -0.00253,1.81076 -0.00253,2.75876c0,0.94801 0,1.74632 0,2.58676c0,0.81236 0.04697,1.78174 -0.00076,2.70073c-0.04694,0.90356 -0.45071,1.67355 -0.55002,2.57579c-0.10562,0.95973 0.11478,1.88283 -0.04385,2.68762c-0.16418,0.83281 -0.42822,1.73192 -0.65485,2.56734c-0.24023,0.88554 -0.6394,1.66241 -0.84076,2.4075c-0.25888,0.95776 -0.11914,1.91278 -0.16052,2.74803c-0.04697,0.94835 -0.38599,1.66208 -0.5296,2.65076c-0.12662,0.87172 -0.03214,1.5872 -0.03214,2.67622c0,0.77457 0,1.8055 0,2.61252c0,0.89926 0,1.79926 0,2.69925c0,0.89998 0,1.752 0,2.70073c0,0.81639 0,1.77902 0,2.69925c0,0.84375 -0.00455,1.74376 0,2.58751c0.00507,0.9436 0.6478,0.66766 1.16016,0.1781c0.60263,-0.57581 1.3801,-1.36838 1.93359,-1.92188c0.5535,-0.5535 1.3425,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19534,-1.19531 1.97745,-1.97745c0.54208,-0.54208 1.0954,-1.0954 1.93661,-1.93661c0.56027,-0.56024 1.10699,-1.10699 1.62674,-1.62674c0.47925,-0.47925 1.42203,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49881,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87703 2.14063,-1.43587c0.68152,-0.49742 1.39441,-1.03616 2.05167,-1.6875c0.66702,-0.66113 1.25513,-1.25514 1.88281,-1.88287c0.62698,-0.62697 1.58316,-2.26659 2.0625,-1.59375c0.49631,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55844,0.6461 -1.14902,1.38821 -1.56601,1.875c-0.77551,0.90541 -1.31378,1.46774 -2.09024,2.25c-0.52145,0.52545 -1.20465,1.23346 -1.80469,1.79298c-0.71036,0.66237 -1.27664,1.08246 -1.86057,1.54903c-0.90967,0.72688 -1.45654,1.26782 -2.29041,1.92668c-0.57996,0.45822 -1.32983,1.05643 -2.10684,1.57506c-0.66531,0.44409 -1.52319,0.73419 -2.26865,1.30577c-0.68564,0.5257 -1.28015,1.23956 -1.94476,1.87387c-0.5206,0.49686 -1.35648,0.95886 -2.09109,1.55139c-0.67581,0.54509 -1.04337,1.41599 -1.92599,1.85776c-0.8183,0.40958 -1.8374,0.38681 -2.65622,0.13779c-0.83701,-0.25458 -1.39453,-1.10156 -1.94528,-1.65233c-0.71085,-0.71083 -1.28992,-1.33398 -2.00452,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13223,-1.60052c-0.65433,-0.49887 -1.39615,-1.0434 -2.08594,-1.60547c-0.80875,-0.659 -1.47189,-1.09024 -2.20407,-1.54901c-0.48975,-0.30692 -1.39172,-0.862 -2.14801,-1.35825c-0.85925,-0.56393 -1.57953,-1.05087 -2.22891,-1.5c-0.81497,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72751,-0.49817 -1.08914,-0.75778 -2.20258,-1.42647c-0.69907,-0.41983 -1.70813,-0.95108 -2.33432,-1.26852c-0.85785,-0.43491 -1.63159,-0.77745 -2.42627,-1.23824c-0.875,-0.50749 -1.62817,-1.06126 -2.39539,-1.45494l-0.73959,-0.34584" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="4" stroke="#000000" fill="none"/><text style="cursor: move;" id="svg_24" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="407.21526" x="187.11445" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#ff0000">30</text><text style="cursor: move;" id="svg_26" transform="matrix(1.47561, 0, 0, 1.52941, -39.3864, -132.419)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="287.53051" x="231.25317" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#007f00">1 1 1 1</text><path id="svg_27" d="m305.82568,326.41013c-0.0321,1.91251 -0.31989,2.69525 -0.4942,3.7366c-0.12848,0.76743 -0.14734,1.79764 -0.3075,2.54315c-0.1499,0.6976 -0.30872,1.70767 -0.3233,2.72025c-0.00977,0.68097 -0.08557,1.78275 -0.2392,2.60178c-0.11865,0.63232 -0.29688,1.62439 -0.32074,2.51447c-0.02716,1.01218 -0.00256,1.81076 -0.00256,2.75876c0,0.94806 0,1.74637 0,2.58676c0,0.81238 0.047,1.78174 -0.00073,2.70074c-0.04694,0.90356 -0.45074,1.67355 -0.55005,2.57578c-0.10559,0.95978 0.11481,1.88284 -0.04382,2.68762c-0.16418,0.83282 -0.42822,1.73193 -0.65485,2.56735c-0.24023,0.88553 -0.6394,1.66241 -0.84076,2.4075c-0.25891,0.95776 -0.11914,1.91278 -0.16052,2.74802c-0.047,0.94836 -0.38599,1.66208 -0.5296,2.65076c-0.12665,0.87173 -0.03217,1.58722 -0.03217,2.67624c0,0.7746 0,1.80548 0,2.61255c0,0.89926 0,1.79926 0,2.69925c0,0.89993 0,1.75195 0,2.70068c0,0.81644 0,1.77902 0,2.69925c0,0.84375 -0.00452,1.74377 0,2.58752c0.00507,0.9436 0.64783,0.66766 1.16016,0.1781c0.60266,-0.57581 1.38013,-1.36838 1.93359,-1.92188c0.55353,-0.5535 1.34253,-1.3425 1.99219,-1.99219c0.46124,-0.46124 1.19537,-1.19531 1.97748,-1.97745c0.54205,-0.54208 1.0954,-1.0954 1.93658,-1.93661c0.5603,-0.56024 1.10699,-1.10699 1.62677,-1.62674c0.47925,-0.47925 1.422,-1.42651 1.93353,-1.9335c0.6673,-0.66144 1.38019,-1.20023 2.06171,-1.69763c0.76575,-0.55881 1.49878,-1.09619 2.36542,-1.54837c0.6438,-0.33591 1.37488,-0.87704 2.14063,-1.43588c0.68152,-0.49741 1.39441,-1.03616 2.05157,-1.6875c0.66711,-0.66113 1.25513,-1.25513 1.88287,-1.88287c0.62701,-0.62692 1.58319,-2.26654 2.0625,-1.59375c0.49628,0.69659 -1.10596,1.20221 -1.6875,1.875c-0.55847,0.64615 -1.14899,1.38821 -1.56598,1.875c-0.77557,0.90546 -1.31384,1.46774 -2.09027,2.25c-0.52148,0.52545 -1.20465,1.23346 -1.80469,1.793c-0.71033,0.66235 -1.27661,1.08249 -1.86053,1.54901c-0.90967,0.72693 -1.45654,1.26782 -2.29041,1.92673c-0.57996,0.45822 -1.32983,1.05643 -2.10687,1.57501c-0.66528,0.44409 -1.52319,0.73419 -2.26862,1.30582c-0.68567,0.5257 -1.28015,1.23953 -1.94476,1.87387c-0.52063,0.49686 -1.35651,0.95883 -2.09113,1.55139c-0.67578,0.54504 -1.04333,1.41599 -1.92596,1.85776c-0.8183,0.40958 -1.83746,0.38681 -2.65625,0.13779c-0.83698,-0.25458 -1.39453,-1.10156 -1.94531,-1.65234c-0.71082,-0.71082 -1.28986,-1.33398 -2.00446,-1.87994c-0.66058,-0.50473 -1.4776,-1.10141 -2.13226,-1.60052c-0.6543,-0.49887 -1.39612,-1.0434 -2.08594,-1.60547c-0.80872,-0.659 -1.47192,-1.09024 -2.20404,-1.54901c-0.48975,-0.30692 -1.39178,-0.862 -2.14801,-1.35825c-0.85931,-0.56393 -1.57953,-1.05087 -2.229,-1.5c-0.81494,-0.5636 -1.45013,-1.02014 -2.14941,-1.49899c-0.72748,-0.49817 -1.08911,-0.75778 -2.20251,-1.42648c-0.69904,-0.41983 -1.70813,-0.95108 -2.33429,-1.26852c-0.85785,-0.43491 -1.63165,-0.77744 -2.42627,-1.23825c-0.87506,-0.50748 -1.62823,-1.06125 -2.39539,-1.45493l-0.73962,-0.34583" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="4" stroke="#000000" fill="none"/><text id="svg_28" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="408.69028" x="232.12581" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#ff0000">36</text><text id="svg_1" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="284.95478" x="157.81656" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#000000">3</text><text id="svg_2" transform="matrix(1.74074, 0, 0, 1.43137, -63.2176, -155.699)" xml:space="preserve" text-anchor="middle" font-family="serif" font-size="24" y="284.9548" x="270.98683" stroke-linecap="null" stroke-linejoin="null" stroke-dasharray="null" stroke-width="0" stroke="#000000" fill="#000000">9</text></g></svg>

It might still be possible to adapt it and remain computationally faster (atleast the additional numbers on the first pass are only two per predicted sample, alot of the computations are still duplicated). But it is not clear to me how to do that properly. Instead I propose to ignore the problem :) 

The reason is that for the paper, we are only evaluating trialwise, so we are not looking at samplewise accuracies anyways. So I could still do the same transformation (for Residual nets) and train the net this way. The net would still be somewhat forced to predict the same label for different input windows within one trial. Only now, to predict a sample, the net will also look a bit into the future (but that should be ok since we are evaluating trialwise).

Other option would be to train the "normal" way, i.e. cut out a new window per sample, but that might simply be too slow.